In [244]:
from selenium import webdriver
import os,json,tqdm,pickle
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, WebDriverException

from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re


In [9]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\python-projects\\Danawa-Auto'

우선 월별 판매실적을 수집하자

In [16]:
params = {'Work':'record','Tab':'Model','Month':None,'MonthTo':None,'Classify':None}

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [18]:
params

{'Classify': None,
 'Month': None,
 'MonthTo': None,
 'Tab': 'Model',
 'Work': 'record'}

In [70]:
#페이지 내 판매실적을 가져와 전역변수에 저장하는 get_car_sold 함수
def get_car_sold(soup):
    global to_json, exceptions,y,m
    
    # 판매량 테이블 받아온다
    car_solds = soup.select('#autodanawa_gridC > div.gridMain > article > main > div > table > tbody > tr')
    
    # 브랜드, 세그먼트, 모델, 타이틀, 연월로 구성된 키를 만든다
    # 밸류로 모델판매량(total)과 라인업별 판매량(존재시) 가진다
    
    for car_sold in car_solds:
        if car_sold.attrs == {}: # model
            brand = car_sold.find('input')['brand']
            model = car_sold.find('td',{'class':'title'}).find('a')['href'][-4:]
            title = car_sold.find('input')['title']

            key = brand + '_' + params['Classify'] + '_'  + model + '_' + title + '_{year}{month}'.format(year=y,month=m)

            to_json[key] = {}
            to_json[key]['total'] = int(re.sub('[\D]','',car_sold.find('td',{'class':'num'}).text))

        elif car_sold.attrs['name'] == 'sub_model': # submodel
            if 'total' in car_sold.attrs['class']:
                continue
            sub_title = car_sold.find('td',{'class':'title'}).text.strip()
            sub_num = int(car_sold.find('td',{'class','num'}).text.replace(',','').strip())
            to_json[key][sub_title] = sub_num

        else:
            print('~~!~!')
            pass

            
            
            
            
            

In [71]:
params

{'Classify': 'O~',
 'Month': '2018-12-00',
 'MonthTo': None,
 'Tab': 'Model',
 'Work': 'record'}

In [100]:
if __name__ == '__main__':
    to_json = {}
    exceptions = []
    
    for y in tqdm.tqdm(range(2011,2019)): # 2011.12 ~ 2018.12
        for m in range(1,13):
            url = 'http://auto.danawa.com/newcar'
            if m<10:
                m = str(0)+str(m)
                params['Month'] = '{}-{}-00'.format(y,m)
            else:
                m = str(m)
                params['Month'] = '{}-{}-00'.format(y,m)
                
            seg_lst = ['~C','PC1','PC2','PC3','PC4','PC5','P5~','RB~','RU2','RU3','RU5','RM~','O~']
            for seg in seg_lst:
                params['Classify'] = seg
            
                try:
                    res = requests.get(url,params=params,headers=headers,timeout=10)
                    res.encoding = 'utf-8'
                    soup = BeautifulSoup(res.text,'lxml')
                    
                    get_car_sold(soup)
                
                except requests.exceptions.Timeout:
                    exceptions.append((str(y)+str(m),seg))
                    print("Timeout occurred")
                
                except Exception as e:
                    exceptions.append((str(y)+str(m),seg))

In [97]:
len(to_json)

15839

In [98]:
len(exceptions)

7

In [99]:
# timeout Error 난 params에 대해 데이터 따로 업데이트 해줌
for e in exceptions:
    y = e[0][:4]
    m = e[0][4:]
    seg = e[1]
    
    url = 'http://auto.danawa.com/newcar'

    params['Month'] = '{}-{}-00'.format(y,m)
    params['Classify'] = seg

    try:
        res = requests.get(url,params=params,headers=headers,timeout=10)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text,'lxml')

        get_car_sold(soup)

    except requests.exceptions.Timeout:
        print(e)
        print("Timeout occurred")

    except Exception as e1:
        print(e1)

In [101]:
# pandas 로 쌓기
model_pd = {'brand':[],'seg':[],'model':[],'YM':[],'title':[],'sold':[]}
sub_model_pd =  {'brand':[],'seg':[],'model':[],'YM':[],'title':[],'sub_title':[],'sub_sold':[]}
for k,v in to_json.items():
    splited = k.split('_')
    for sub_k,sub_v in v.items():
        if sub_k == 'total':
            continue
        sub_model_pd['brand'].append(splited[0])
        sub_model_pd['seg'].append(splited[1])
        sub_model_pd['model'].append(splited[2])
        sub_model_pd['YM'].append(splited[4])
        sub_model_pd['title'].append(splited[3])
        sub_model_pd['sub_title'].append(sub_k)
        sub_model_pd['sub_sold'].append(sub_v)
        
    model_pd['brand'].append(splited[0])
    model_pd['seg'].append(splited[1])
    model_pd['model'].append(splited[2])
    model_pd['YM'].append(splited[4])
    model_pd['title'].append(splited[3])
    model_pd['sold'].append(v['total'])

In [102]:
mon_sub = pd.DataFrame(sub_model_pd)
mon = pd.DataFrame(model_pd)

In [103]:
mon_sub.to_csv('monthly_sub.csv',encoding='cp949')
mon.to_csv('monthly.csv',encoding='cp949')

In [8]:
mon_sub.to_csv('monthly_sub_utf.csv',encoding='utf-8')
mon.to_csv('monthly_utf.csv',encoding='utf-8')

In [104]:
mon.sample(5) # model 단위 판매실적

,brand,seg,model,YM,title,sold
5636,546,RU3,2508,201408,SRX,4
2861,326,PC5,2041,201305,체어맨 H 뉴 클래식,171
118,362,RU3,2371,201112,X3,53
14892,573,PC5,2955,201808,Continental,77
7978,509,RU3,3158,201508,QX70,2


In [105]:
mon_sub.sample(5) # trim (sub_model) 단위 판매실적

,brand,seg,model,YM,title,sub_title,sub_sold
5042,367,RU2,2445,201512,Countryman,Cooper D ALL4,107
19141,307,~C,3333,201810,모닝,1.0 VAN,228
16868,349,PC5,3511,201804,더 뉴 S-class,S560 4MATIC Long,146
1977,371,RU2,2423,201505,Q3,30 TDI quattro,23
2331,349,PC5,3128,201506,S-class,S600,6


In [106]:
mon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15839 entries, 0 to 15838
Data columns (total 6 columns):
brand    15839 non-null object
seg      15839 non-null object
model    15839 non-null object
YM       15839 non-null object
title    15839 non-null object
sold     15839 non-null int64
dtypes: int64(1), object(5)
memory usage: 742.5+ KB


차량 데이터(기본정보, 제원, 사양옵션 등)와 중고차 데이터 수집

In [142]:
path = './chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')

In [143]:
driver = webdriver.Chrome(path,chrome_options=options)

In [ ]:
os.chdir('./data')

In [7]:
# # 판매실적 데이터에서 모델아이디를 추출 
# model_ids = [str(el) for el in list(pd.read_csv('monthly_utf.csv').model.unique())]

In [124]:
# 판매실적 데이터에서 모델아이디를 추출  SUV
data = pd.read_csv('monthly_utf.csv')
model_ids = [str(el) for el in list(data[data.seg=='RU2'].model.unique())]

In [126]:
#params 재정의
params = {}
params['Work'] = 'model'
params['Model'] = None

In [127]:
# model_id가 인덱스인 pd 형태로 저장
car_base = {'model':[],'brand':[],'seg':[],'title':[],'launching':[],'price_range':[],'recall_num':[],'recall_txt':[]}

In [128]:
# car_base의 데이터를 채우는 함수 
def get_car_base(model_id):
    global car_base, params
    
    url = 'http://auto.danawa.com/auto/?attributeList='
    params['Model'] = str(model_id)
    
    for k,v in params.items():
        url += '&'+k+'='+v
    driver.get(url)
    
    # 기본정보 수집
    car_base['model'].append(model_id)
    
    category = driver.find_element_by_css_selector('#autodanawa_gridC > div.gridLeft > ul')
    car_base['brand'].append(category.find_element_by_class_name('depth3').text)
    car_base['seg'].append(category.find_element_by_class_name('depth4').text)
    
    info = driver.find_element_by_class_name('info')
    car_base['title'].append(info.find_element_by_class_name('title').text)
    car_base['launching'].append(info.find_element_by_class_name('date').text)
    
    car_base['price_range'].append(re.sub(r'[가-힣ㄱ-ㅎa-zA-Z|,|\s]','',driver.find_element_by_class_name('price').text.strip()))
    
    #리콜탭
    driver.find_element_by_link_text('보증/리콜').click()
    
    recall_lst = driver.find_elements_by_class_name('recallList')
    car_base['recall_num'].append(len(recall_lst))
    
    recall_txt = []
    for recall in recall_lst:
        recall_subj = recall.find_element_by_class_name('subj')
        recall_txt.append(recall_subj.find_element_by_class_name('cate').text.strip()+'_'+recall_subj.find_element_by_class_name('title').text.strip())
    car_base['recall_txt'].append(recall_txt)

In [1]:
if __name__=='__main__':
    base_exceptions=[]
    for model_id in tqdm.tqdm(model_ids):
        try:
            get_car_base(model_id)
        except Exception as e:
            print(e)
            base_exceptions.append(model_id)

In [ ]:
car_base = pd.DataFrame(car_base)

In [ ]:
car_base.to_csv('car_base_recall_utf.csv',encoding='utf-8')

In [ ]:
car_base.sample(5)

이후 모바일 웹에서 진행

In [70]:
path = './chromedriver.exe'

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('disable-gpu')

driver = webdriver.Chrome(path,chrome_options=options)

이제 trim 단위로 차량 정보 가져오자

In [233]:
def get_car_detail(model_id):
    global params
    tmp_detail = {'model':[],'lineup':[],'trim':[],'price':[],'spec':[],'item':[]} 
    
    url = 'http://m.auto.danawa.com/auto/?Work=model'
    params['Model'] = str(model_id)
    for k,v in params.items():
        if k!= 'Work':
            url += '&'+k+'='+v
    
    driver.get(url)
    
    # price tab
    params['Tab'] = 'price'
    driver.find_element_by_class_name('last').find_element_by_tag_name('button').click()
    driver.find_element_by_link_text('가격').click()
    
    # 라인업 리스트 가져와 lineup_cds에 저장
    driver.find_element_by_id('buttonSelect').click()

    lineup_selc = driver.find_element_by_id('popup_data')
    trim_infos = lineup_selc.find_elements_by_class_name('trimInfo')

    lineup_lst = []
    for trim_info in trim_infos:
        for li in trim_info.find_elements_by_tag_name('li'): # onSale포함 모든 라인업 가져옴
            lineup_lst.append(li)
#             print(li.text) ###########
            
    driver.back()
    
    lineup_cds = []
    for lu in lineup_lst:
#             print(lu.find_element_by_class_name('name').text.strip())
        lu_href = lu.find_element_by_tag_name('a').get_attribute('href')
        lu_idx = lu_href.find('&Lineup=')
        lineup_cds.append(lu_href[lu_idx+8:lu_idx+13])
    driver.back()
    
    # 라인업을 돌며 trim별 정보 가져오기
    for lu_cd in lineup_cds:
        params['Lineup'] = lu_cd
        url = 'http://m.auto.danawa.com/auto/?Work=model'
        url += '&Model='+params['Model']
        url += '&Lineup='+params['Lineup']
        url += '&Tab='+'price'
        
        driver.get(url) # 라인업으로 get url

        trims = driver.find_element_by_id('priceList').find_elements_by_tag_name('dt') # trim 수 만큼 dt
        for trim in trims:

            tmp_detail['model'].append(model_id)
            tmp_detail['lineup'].append(lu_cd)
            try:
                tmp_detail['trim'].append(trim.find_element_by_class_name('title').text.strip())
                tmp_detail['price'].append(trim.find_element_by_class_name('price').text.strip())
            except Exception:
                print('No trim or No price')
                tmp_detail['trim'].append('err')
                tmp_detail['price'].append('err')
                continue
                    
    # 제원 tab
    # 라인업을 돌며 trim별 정보 가져오기
    for lu_cd in lineup_cds:
        params['Lineup'] = lu_cd
        url = 'http://m.auto.danawa.com/auto/?Work=model'
        url += '&Model='+params['Model']
        url += '&Lineup='+params['Lineup']
        url += '&Tab='+'spec'
        
        driver.get(url) # 라인업으로 get url
        
        buttonSelectSub = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.ID, "buttonSelectSub")))
        buttonSelectSub.click()
        
        popup_data = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.ID, "popup_data")))
        
        trim_buttons = popup_data.find_element_by_class_name('lineupList').find_elements_by_tag_name('li')
        trims_len = len(trim_buttons) # stale Element Error 때문에 우선 trim 길이만 가져옴
        driver.back()
        
        for button_idx in range(trims_len):
#             print(trims_len,'중',button_idx)
            try:
                buttonSelectSub = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.ID, "buttonSelectSub")))
                
                buttonSelectSub.click()
#                 driver.implicitly_wait(2)
                
                popup_data = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.ID, "popup_data")))
                
                trim_buttons = popup_data.find_element_by_class_name('lineupList').find_elements_by_tag_name('li')
                
#                 trim_buttons[button_idx].click() # 인덱스로 접근
                trim_buttons[button_idx].find_element_by_tag_name('button').click() # 인덱스로 접근
                
                spec_lst_body = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "#autodanawa_content > article > main > div.modelSection.selectBar > table > tbody")))
                spec_lst = spec_lst_body.find_elements_by_tag_name('tr')
#                 spec_lst = driver.find_elements_by_css_selector('#autodanawa_content > article > main > div.modelSection.selectBar > table > tbody > tr')
    
                tmp_spec = {}
                for tr in spec_lst:
                    tmp_spec[tr.find_element_by_class_name('tdTitle').text] = tr.find_elements_by_class_name('tdCenter')[button_idx].text
                
                print('success',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
    
            except WebDriverException as we:
                tmp_spec = {}
                print('WebDriverException',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
        
            except Exception as e:
                tmp_spec = {}
                print('Exception',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
            
            finally:
                tmp_detail['spec'].append(tmp_spec)
     
    # 사양/옵션 tab
    # 라인업을 돌며 trim별 정보 가져오기
    for lu_cd in lineup_cds:
        params['Lineup'] = lu_cd
        url = 'http://m.auto.danawa.com/auto/?Work=model'
        url += '&Model='+params['Model']
        url += '&Lineup='+params['Lineup']
        url += '&Tab='+'item'
        
        driver.get(url) # 라인업으로 get url
        
        buttonSelectSub = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.ID, "buttonSelectSub")))
        buttonSelectSub.click()
        
        popup_data = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.ID, "popup_data")))
        
        trim_buttons = popup_data.find_element_by_class_name('lineupList').find_elements_by_tag_name('li')
        trims_len = len(trim_buttons) # stale Element Error 때문에 우선 trim 길이만 가져옴
        driver.back()
        
        for button_idx in range(trims_len):
#             print(trims_len,'중',button_idx)
            try:
                buttonSelectSub = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(
                        (By.ID, "buttonSelectSub")))
                
                buttonSelectSub.click()
#                 driver.implicitly_wait(2)
                
                popup_data = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.ID, "popup_data")))
                
                trim_buttons = popup_data.find_element_by_class_name('lineupList').find_elements_by_tag_name('li')
                
#                 trim_buttons[button_idx].click() # 인덱스로 접근
                trim_buttons[button_idx].find_element_by_tag_name('button').click() # 인덱스로 접근
                
                spec_lst_body = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "#autodanawa_content > article > main > div.modelSection.selectBar > table > tbody")))
                spec_lst = spec_lst_body.find_elements_by_tag_name('tr')
#                 spec_lst = driver.find_elements_by_css_selector('#autodanawa_content > article > main > div.modelSection.selectBar > table > tbody > tr')
    
                tmp_item = {}
                for i,tr in enumerate(spec_lst):
                    tmp_item[tr.find_element_by_class_name('tdTitle').text+'_'+str(i)] = tr.find_elements_by_class_name('tdCenter')[button_idx].text
                
                print('success',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
    
            except WebDriverException as we:
                tmp_item = {}
                print('WebDriverException',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
        
            except Exception as e:
                tmp_item = {}
                print('Exception',params['Lineup'],button_idx,trim_buttons[button_idx].text)
                continue
            
            finally:
                tmp_detail['item'].append(tmp_item)
    
                
    return tmp_detail

In [249]:
if __name__=='__main__':
    #params 재정의
    params = {}
    params['Work'] = 'model'
    params['Model'] = None
    
    car_detail = {'model':[],'lineup':[],'trim':[],'price':[],'spec':[],'item':[]} 
    detail_exceptions = []
    
    for i, model_id in enumerate(model_ids):
        if i%1 == 0:
            with open('car_detail_{}.dict'.format(i),'wb') as fp:
                pickle.dump(car_detail,fp)
            car_detail = {'model':[],'lineup':[],'trim':[],'price':[],'spec':[],'item':[]} 
        elif i == len(model_ids)-1:
            with open('car_detail_{}.dict'.format(i),'wb') as fp:
                pickle.dump(car_detail,fp)
            car_detail = {'model':[],'lineup':[],'trim':[],'price':[],'spec':[],'item':[]} 
        else:
            pass
            
        try:
            tmp_detail = get_car_detail(model_id)
            for k,v in tmp_detail.items():
                car_detail[k].extend(v)
                
        except Exception as e:
            print('skip model',model_id)
            detail_exceptions.append(model_id)
            continue
        

In [250]:
car_detail_df = pd.DataFrame(car_detail)

In [247]:
driver.quit()

모델id와 매핑시킬 중고차 번호를 가져오자

In [16]:
os.chdir('./..')

In [25]:
path = './chromedriver.exe'

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('disable-gpu')

In [26]:
driver = webdriver.Chrome(path,options=options)

In [27]:
# model과 중고차id를 매핑
# json 형태로 저장 model_id 가 키
model_used = {}

In [33]:
# params 재정의
params = {}
params['Model'] = None

In [34]:
def get_used_no(model_id):
    global model_used, params
    params['Model'] = str(model_id)
    params['Page'] = None
    
    model_used[str(model_id)] = []
    
    max_page = 20
    for p in range(max_page):
        url = 'http://auto.danawa.com/usedcar/?Tab=list'
        params['Page'] = str(p+1)
        for k,v in params.items():
            url += '&'+k +'=' + v
        print(url)
        driver.get(url)
        
        if driver.find_element_by_class_name('total').find_element_by_class_name('num').text == '0': # 해당 페이지에 매물x
            break
            
        used_car_lst = driver.find_element_by_id('usedcarList').find_elements_by_class_name('row')

        for used in used_car_lst:
            href = used.find_element_by_class_name('name').get_attribute('href')
            parsed = urlparse(href).query.split('&')
            for kv in [el.split('=') for el in parsed]: # pased 된 리스트에서 sDemoNo 밸류 찾기
                if kv[0] == 'sDemoNo':
                    model_used[model_id].append(kv[1])

In [37]:
used_exceptions = []
for model_id in tqdm.tqdm(model_ids):
    get_used_no(model_id)
    try:
        pass
    except Exception as e:
        used_exceptions.append(model_id)
        print(e)

In [38]:
model_used['2132'][:5]

['9120002093', '9110002001', '9110001297', '9011002014', '8910000607']